# 02 - Supervised Learning - Regression

## Objectives:
1. Demonstrate the workflow of a machine learning problem.

Quick recap
-----------
1. **Supervised learning** is a category of machine learning which focus on getting accurate prediction (class / number), given training data (a matrix of examples and desired output).
2. If the desired output (called "label" or "target" in machine learning terminology) is a continous number, e.g. price, temperature, stock index... then it is called a **Regression** problem
3. Usually, the data is given in as a n \* m matrix, where each row is an example/sample and each column is a property of the sample (called *feature*).

In [2]:
# Load some packages and settings

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import pandas as pd
import os

# to make this notebook's output stable across runs
rnd = 1
np.random.seed(rnd)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "1"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

print("Hello World!")

Hello World!


Let's load and take a look at our data again.

# 1. Get data

In [3]:
HOUSING_PATH = os.path.join("data", "housing")  # path of the data

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)
housing = load_housing_data()  # Load data
print(housing.shape)

(20640, 10)


In [4]:
# Print the first 5 rows of data
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


The matrix `housing` now holds our data - you can see each row is one example, and each column describes the example.

Since we are to predict, the `median_house_value`, let's take it out so the features `longitude`, `median_income` etc. are stored in a variable X,

and the target `median_house_value` stored in variable y.

In [5]:
X = housing.drop('median_house_value', axis=1)  # the features
y = housing['median_house_value']  # the target

# 2. Training set, Validation set and Test set

Before we dive in, there is a very important step to do: let's split the data set into 2, one for training the model (commonly called the * *training set* *) and the other for evaluation (the * *test set* *).

This is **VERY IMPORTANT** in modeling - we need to hold out a set of data, which is not known by the model, to evaluate how well the model perform.
This data set is called the **test set**.

An analogue, in school - we are *__trained__* with materials in lessons (reading, lectures...). To evaluate how well we understand the materials, we are *__tested__* in with materials that are relevant, but not known in advance (i.e. examination). Otherwise we will simply recite (* *memorize* *) the questions and answers in exam and get a good score, without actually understand the materials at all!

### Question: what problem will it pose if a model has access to the test data during training?

In [6]:
# Split the data into train and test set
from sklearn.model_selection import train_test_split
test_size = 0.1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=rnd)

In [7]:
print(X_train.shape[0], X_test.shape[0])

18576 2064


From now on, we will use `X_train` and `y_train` for training the model (i.e. tune the parameters of the model), and **leave the test set `X_test` `y_test` untouched** until we are done with modelling.

Again, **NEVER TUNE YOUR MODEL WITH THE TEST SET!** It should only be used for final evaluation.

To start simple, let's pick **1** feature with highest correlation to the target. 

In [8]:
# Compute correlation, take absolute, and sort in descending order
abs(X_train.corrwith(y_train)).sort_values(ascending=False)  # the highest one is "median_income"

median_income         0.687827
latitude              0.141070
total_rooms           0.134944
housing_median_age    0.102239
households            0.065731
total_bedrooms        0.050145
longitude             0.047969
population            0.025464
dtype: float64

In [9]:
# pick `median_income` as the only feature
X_train_1 = X_train['median_income']

In [10]:
# define the error metric (in this case the mean squared error, MSE)
from sklearn.metrics import mean_squared_error

# pick a model
from sklearn.linear_model import LinearRegression

X_train_1 = np.array(X_train_1).reshape(-1, 1)  # preparing feature (data X)
#train_set_univariate__y = train_set_univariate['median_house_value']  # preparing the target (y)

linear_reg = LinearRegression()
linear_reg.fit(X=X_train_1, y=y_train)  # feed X and y into the linear model, ask it to 'train'

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [11]:
print("Slope `m`: {:,.2f}\nIntercept `c`: {:,.2f}".format(linear_reg.coef_[0], linear_reg.intercept_))

Slope `m`: 41,754.99
Intercept `c`: 45,400.46


In [12]:
# get predictions for validation set
y_pred_train = linear_reg.predict(np.array(X_train_1).reshape(-1, 1))

# evaluate with the Validation set
from sklearn import metrics

def print_eval(y_true, y_pred):
    print('Average target in train set: {:,.0f}'.format(y_true.mean()))
    print('Mean Absolute Error on train set: {:,.0f}'.format(metrics.mean_absolute_error(y_true, y_pred)))
    print('Root Mean Squared Error on train set: {:,.0f}'.format(np.sqrt(metrics.mean_squared_error(y_true, y_pred))))
    print('Relative Root Mean Squared Error on train set: {:.2f} %'.format(100 * np.sqrt(metrics.mean_squared_error(y_true, y_pred))/y_true.mean()))
    print('R^2 of train set: {:,.2f}'.format(metrics.r2_score(y_true, y_pred)))
    return None

print("Error in training set:")
print_eval(y_train, y_pred_train)

Error in training set:
Average target in train set: 207,202
Mean Absolute Error on train set: 62,644
Root Mean Squared Error on train set: 83,711
Relative Root Mean Squared Error on train set: 40.40 %
R^2 of train set: 0.47


You see that the error (RMSE) is quite high (84K, or 40.4%). How can we do better?

# 3. Improve model performance

There are many ways to optimize a model's performance:
1. Try different models
2. Build an ensemble of models
3. Better feature engineering
4. Obtain more data

Let's look at these approaches.

## 3.1: Try different models

There are a lot of models frequently come up in machine learning, one of them being the family of **Decision Tree**.

Let's try on our problem.

In [13]:
# import decision tree
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()

# train the model on 1 feature
tree_reg.fit(X_train_1, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [14]:
# evaluate on train set
y_pred_train = tree_reg.predict(X_train_1)
y_pred_validate = tree_reg.predict(X_train_1)

# evaluate on validation sete
print("Error in training set:")
print_eval(y_train, y_pred_train)

Error in training set:
Average target in train set: 207,202
Mean Absolute Error on train set: 25,362
Root Mean Squared Error on train set: 48,588
Relative Root Mean Squared Error on train set: 23.45 %
R^2 of train set: 0.82


At first glance, the decision tree model is performing great - it reaches an RMSE of ~49K on train set, which is much less than the error of the linear regression model (84K). Does this suggest the decision tree is a better fit for the problem?

Not at all! If we evaluate the model on new data, we will soon see that the RMSE is much higher (_~104K_) than in training set. This is a sign of the model being severely **overfit**, and likely to **perform bad on future data**.

One popular way to evaluate a model is **cross-validation**, which we will try:

In [15]:
# Cross validation
from sklearn.model_selection import cross_val_score
tree_reg = DecisionTreeRegressor()
scores = cross_val_score(tree_reg, X_train_1, y_train, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores: ", scores)
    print("Mean: {:,.0f}".format(scores.mean()))
    print("Standard deviation: {:,.0f}".format(scores.std()))

display_scores(tree_rmse_scores)

Scores:  [104657.89001527 109623.56773174 108277.63146136 108692.00323851
 107907.16873381 109107.99605406 110096.13864112 105161.92191748
 105958.2773545  106639.09809206]
Mean: 107,612
Standard deviation: 1,805


Cross validation gives an average error of ~108K, which is far more than on the training set (49K). We can expect our decision tree model will give a RMSE of ~108K on future data.

Also note that decision tree performs worse than the simple linear regression problem - can you prove that?

### Exercise: 
1. Use cross validation to build and evaluate a linear regression model. What is the average RMSE?
2. Does the linear regression model overfit? Why?

In [16]:
from sklearn.model_selection import cross_val_score
lin_reg = LinearRegression()
scores = cross_val_score(lin_reg, X_train_1, y_train, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-scores)
display_scores(lin_rmse_scores)

Scores:  [83577.42916677 84437.91218285 83101.60275605 83722.9528075
 86459.93998787 84550.82568437 81990.1916529  83007.38587775
 84477.28858687 81873.13334893]
Mean: 83,720
Standard deviation: 1,288


Now the question: what can we do to prevent overfitting? There are several ways:

1. Obtain more data (again!) 
2. Regularization
3. Tune the hyperparameter(s) of a model with validation set

We will cover more in future.

## 3.2: Build an ensemble of models

Intuition: each model has its own assumption and pros & cons. They make different types of errors.

How about we train a lot of models and combine them together?

The first ensemble model we use is the imfamous Random Forest:

In [17]:
from sklearn.ensemble import RandomForestRegressor
# train the model
forest_reg = RandomForestRegressor(n_estimators=100)
forest_reg.fit(X_train_1, y_train)

# Evaluate on train set
forest_pred = forest_reg.predict(X_train_1)
print_eval(y_train, forest_pred)

# Also Evaluate using cross-validation
scores = cross_val_score(forest_reg, X_train_1, y_train, scoring="neg_mean_squared_error", cv=5)
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)

Average target in train set: 207,202
Mean Absolute Error on train set: 38,974
Root Mean Squared Error on train set: 55,385
Relative Root Mean Squared Error on train set: 26.73 %
R^2 of train set: 0.77
Scores:  [95499.71334037 98435.11170061 97526.85597194 96345.00069502
 95472.1373001 ]
Mean: 96,656
Standard deviation: 1,163


Again, the training error (55K) is much lower than on validation set (average 97k), indicating overfitting. However, it is better than decision tree.

## 3.3: Better feature engineering

In previous attempts, we used only 1 feature, the `median_income`. Obviously we have more than that - for example `latitude` and `total_rooms`. Will they give our models a boost?

Let's try a no-brainer's approach - feed all avaiable features into the models and see how it goes.

In [18]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

num_pipeline = Pipeline([
('imputer', SimpleImputer(strategy="median")),
('std_scaler', StandardScaler()),
])

X_train_2 = X_train.drop("ocean_proximity", axis=1)  # drop ocean_proximity as we do not know what to do with text yet
X_train_2 = num_pipeline.fit_transform(X_train_2)

In [19]:
# Linear model, with 5-fold cross validation
linear_reg = LinearRegression()
scores = cross_val_score(linear_reg, X_train_2, y_train, scoring="neg_mean_squared_error", cv=5)
linear_rmse_scores = np.sqrt(-scores)
display_scores(linear_rmse_scores)

Scores:  [70703.5121299  69136.04055233 70473.48048049 68426.90344941
 69931.40885099]
Mean: 69,734
Standard deviation: 848


In [20]:
# Decision Tree model, with 5-fold cross validation
dt_reg = DecisionTreeRegressor()
scores = cross_val_score(dt_reg, X_train_2, y_train, scoring="neg_mean_squared_error", cv=5)
dt_rmse_scores = np.sqrt(-scores)
display_scores(dt_rmse_scores)

Scores:  [71478.9896855  70766.52566779 69600.65442669 69501.3704902
 70206.15310401]
Mean: 70,311
Standard deviation: 741


In [21]:
# Random Forest model, with 5-fold cross validation
rf_reg = RandomForestRegressor(n_estimators=100)
scores = cross_val_score(rf_reg, X_train_2, y_train, scoring="neg_mean_squared_error", cv=5)
rf_rmse_scores = np.sqrt(-scores)
display_scores(rf_rmse_scores)

Scores:  [49611.99457775 50192.20650953 48746.97631491 49079.65178797
 49204.1081918 ]
Mean: 49,367
Standard deviation: 497


See the improvement in RMSE?
1. Linear model: 84K -> 70K
2. Linear model: 108K -> 70K
3. Linear model: 97K -> 49K

Looks great! By simply feeding more features into the model, we significantly improve our model! But veterans like us should not settle here.

Let's try custom-craft some features!

In [22]:
X_train_3 = X_train.copy()
X_train_3["rooms_per_household"] = X_train_3["total_rooms"]/X_train_3["households"]
X_train_3["bedrooms_per_room"] = X_train_3["total_bedrooms"]/X_train_3["total_rooms"]
X_train_3["population_per_household"]=X_train_3["population"]/X_train_3["households"]
#X_train_3.drop(['households', 'total_rooms', 'ocean_proximity'], axis=1, inplace=True)
X_train_3.drop(['ocean_proximity'], axis=1, inplace=True)

In [23]:
num_pipeline = Pipeline([
('imputer', SimpleImputer(strategy="median")),
('std_scaler', StandardScaler()),
])
X_train_3 = num_pipeline.fit_transform(X_train_3)

In [24]:
# Linear model, with 5-fold cross validation
linear_reg = LinearRegression()
scores = cross_val_score(linear_reg, X_train_3, y_train, scoring="neg_mean_squared_error", cv=5)
linear_rmse_scores = np.sqrt(-scores)
print("Linear model:\n")
display_scores(linear_rmse_scores)

# Decision Tree model, with 5-fold cross validation
dt_reg = DecisionTreeRegressor()
scores = cross_val_score(dt_reg, X_train_3, y_train, scoring="neg_mean_squared_error", cv=5)
dt_rmse_scores = np.sqrt(-scores)
print("\nDecision Tree:\n")
display_scores(dt_rmse_scores)

# Random Forest model, with 5-fold cross validation
rf_reg = RandomForestRegressor(n_estimators=100)
scores = cross_val_score(rf_reg, X_train_3, y_train, scoring="neg_mean_squared_error", cv=5)
rf_rmse_scores = np.sqrt(-scores)
print("\nRandom Forest:\n")
display_scores(rf_rmse_scores)

Linear model:

Scores:  [69524.4571517  68557.37806979 69443.0003564  67082.90748632
 69180.52607359]
Mean: 68,758
Standard deviation: 903

Decision Tree:

Scores:  [74662.87003119 71975.9785406  74251.91751703 75535.1100789
 70946.99574858]
Mean: 73,475
Standard deviation: 1,726

Random Forest:

Scores:  [50911.61823286 50424.01212084 50035.26505469 50975.20645513
 51080.4242722 ]
Mean: 50,685
Standard deviation: 396


Hmm, having these new features does not seem to improve our models much for now. However, in reality the major difference between a good and a bad model is how well the feature engineering step is, as we will soon see.

Finally, we can try tuning the hyperparameters of the models and see what combination lead to the best result:

In [25]:
from sklearn.model_selection import GridSearchCV
param_grid = [
{'n_estimators': [50, 100, 150], 'max_features': [2, 4, 6], 'bootstrap': [True, False]},
]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_3, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{'bootstrap': [True, False], 'm

In [26]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
                      max_features=4, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=150,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [27]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

52730.53846386389 {'bootstrap': True, 'max_features': 2, 'n_estimators': 50}
52772.665190864354 {'bootstrap': True, 'max_features': 2, 'n_estimators': 100}
52418.06915752724 {'bootstrap': True, 'max_features': 2, 'n_estimators': 150}
50701.26504097175 {'bootstrap': True, 'max_features': 4, 'n_estimators': 50}
50217.27652248249 {'bootstrap': True, 'max_features': 4, 'n_estimators': 100}
50287.52596649337 {'bootstrap': True, 'max_features': 4, 'n_estimators': 150}
50817.6338655036 {'bootstrap': True, 'max_features': 6, 'n_estimators': 50}
50489.717947683064 {'bootstrap': True, 'max_features': 6, 'n_estimators': 100}
50315.93951622703 {'bootstrap': True, 'max_features': 6, 'n_estimators': 150}
51889.26069221178 {'bootstrap': False, 'max_features': 2, 'n_estimators': 50}
51573.456594402254 {'bootstrap': False, 'max_features': 2, 'n_estimators': 100}
51438.598905660154 {'bootstrap': False, 'max_features': 2, 'n_estimators': 150}
49797.90270790439 {'bootstrap': False, 'max_features': 4, 'n_e

The combination `{'bootstrap': False, 'max_features': 4, 'n_estimators': 150}` performs best with RMSE of 49K. In reality, we will do more searches to seek the best combination.

Summarizing model performance:

| Model      |# features used | Cross-validation RMSE |
| ----------- | ----------- | ----------- |
| Linear      | 1       |84K|
| Decision Tree   | 1        |108K|
| Random Forest   | 1        |97K|
| Linear      | 8       |70K|
| Decision Tree   | 8        |70K|
| Random Forest   | 8        |49K|
| Linear      | 12       |69K|
| Decision Tree   | 12        |73K|
| Random Forest   | 12        |51K|
| Random Forest (with grid search)  | 12        |49K|



The last method, as always, is to get more data (either in volumn or diversity). I will leave this as your exercise.